In [1]:
import os, math, sys
import pandas as pd
import pandapower as pp
import numpy as np

from pandapower.file_io import from_json, to_json

main_dir = os.path.abspath(os.path.join(os.path.abspath(""), ".."))
data_dir = os.path.join(main_dir, 'Modified_116_LV_CSV')

if main_dir not in sys.path:
    sys.path.insert(0, main_dir)

load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\re2.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\vitor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python31

In [2]:
os.path.exists(os.path.join(data_dir, 'profile_datasource.csv'))

True

In [6]:
from copy import deepcopy
import create_basic_network as cbn
# print(cbn.net)
net = deepcopy(cbn.net)
pp.runpp_3ph(net, init='auto', max_iteration=100, tolerance_mva=1e-5, calc_voltage_angles=True, v_debug=True)
net

This pandapower network includes the following parameter tables:
   - bus (117 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (115 elements)
   - trafo (1 element)
 and the following results tables:
   - res_bus_3ph (117 elements)
   - res_line_3ph (115 elements)
   - res_trafo_3ph (1 element)
   - res_ext_grid_3ph (1 element)
   - res_shunt_3ph (1 element)

In [ ]:
pd.read_csv(os.path.join(data_dir, 'profile_datasource.csv'), index_col=0)

In [ ]:
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
loadShape_df = pd.read_csv(os.path.join(data_dir, 'LoadShapes.csv'), skiprows=1, sep=';')
loads_df.head(), loadShape_df.head()

merged_load = loads_df.merge(loadShape_df, left_on="Yearly", right_on="Name", how="left")
merged_load.head(10)

In [ ]:
ts_data = {}

for _, row in merged_load.iterrows():
    # print(row["File"])
    file_path = os.path.join(data_dir, 'Load Profiles', row["File"])
    try:
        profile = pd.read_csv(file_path)
        ts_data[row["Name_x"]] = profile["mult"].values * 1e-3 # or .to_numpy()
    except:
        profile = pd.read_csv(file_path, sep=';')
        ts_data[row["Name_x"]] = profile["mult"].values * 1e-3 # or .to_numpy()
    # finally:
    #     ts_data[row["Name_x"]] = profile["mult"].values * 1e-3 # or .to_numpy()

profile_df = pd.DataFrame(ts_data, index=profile.time)
profile_df.head(10)

In [ ]:
import create_basic_network as cbn

profile_df2 = cbn.create_data_source(data_dir=data_dir, profile_dir=os.path.join(data_dir, 'Load Profiles'))
profile_df2

In [ ]:
from copy import deepcopy
import pandapower.control as ppc

net = cbn.net
ds = cbn.create_data_source(data_dir, profile_dir=os.path.join(data_dir, 'Load Profiles'), ds_index=True)
load_map = {}

for _, row in loads_df.iterrows():
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0

    if (bus_id in net.bus.name) and (row['phases'] in {'A', 'B', 'C'}):
        if row['phases'] == 'A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases'] == 'B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else:  # 'C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))

        load = pp.create_asymmetric_load(
            net, bus=net.bus.index[net.bus.name == bus_id][0],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
        load_map[load] = row['Yearly']

        # Attach ConstControl
        phase = row['phases'].lower()
        ppc.ConstControl(
            net, element_index=load,
            variable=f"p_{phase}_mw",
            element="asymmetric_load",
            profile_name=row['Name'], data_source=ds
        )
        # print(f"Created {net.controller.object.iloc[load+1].profile_name}")

In [ ]:
pp.runpp_3ph(net, max_iteration=1000, tolerance_mva=1e-5)
print("res_bus_3ph:", net.res_bus_3ph.index)

In [ ]:
print(len(profile_df2.df))
profile_df2.df.index

In [7]:
print(f"Trafobus: {cbn.net.trafo.hv_bus.values[0]}, Bus Indexes: {cbn.net.res_bus_3ph.index}")

Trafobus: 0, Bus Indexes: Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116],
      dtype='int64', length=117)


In [ ]:
import pandapower.control as ppc
pq_area = ppc.controller.DERController.PQVAreas.PQArea4105(variant=1)
type(pq_area)

In [ ]:
import pandapower.control as ppc
from pandapower.timeseries.data_sources.frame_data import DFData

net48 = from_json(os.path.join(main_dir, 'json_networks', '48_loads_network.json'))
# net48.asymmetric_load.head(10)

filtered_loads = merged_load[merged_load["Name_x"].isin(net48.asymmetric_load.name.values)]

ds = DFData(profile_df)

for load_name in net48.asymmetric_load.name:
    # get the index of the corresponding load
    idx = net48.asymmetric_load[net48.asymmetric_load.name == load_name].index[0]

    # find phase from the merged metadata
    phase = filtered_loads[filtered_loads["Name_x"] == load_name]["phases"].values[0].lower()

    ppc.ConstControl(
        net48, element_index=idx,
        variable=f"p_{phase}_mw",
        element=f"asymmetric_load",
        profile_name=net48.asymmetric_load.name.at[idx], data_source=ds
    )

    
net48.controller.head(10)

In [ ]:
output_dir = os.path.join(main_dir, "timeseries_results")

if not os.path.exists(output_dir): os.mkdir(output_dir)

In [ ]:
def get_kwarg(a, **kwargs):
    ds_index = kwargs.get('ds_index')
    ds_index = False if ds_index is None else ds_index
    print(a, 'ds_index =', ds_index)
get_kwarg(0, ds_index=True)

In [ ]:
from pandapower.timeseries import run_timeseries
from pandapower.timeseries import OutputWriter

net48 = from_json(os.path.join(main_dir, 'json_networks', '48_loads_network.json'))
net48

# Create and attach output writer to net48
# ow48 = OutputWriter(net48, time_steps=range(profile_df.shape[0]), output_path=output_dir)
# # Log desired variables
# ow48.log_variable('res_trafo_3ph', 'loading_percent', index=net48.trafo.index[0])

# ow48.log_variable('res_bus_3ph', 'vm_b_pu', index=[32, 26]) # LOAD2 and LOAD23
# ow48.log_variable('res_bus_3ph', 'vm_a_pu', index=[14]) # LOAD55

run_timeseries(net48, time_steps=range(profile_df.shape[0]), run=pp.runpp_3ph, continue_on_divergence=False)

In [ ]:
net = from_json(os.path.join(main_dir, 'json_networks', 'no_load_network.json'))

# Create Loads
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
# print(loads_df.head(10))

net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
load_map = {}
for _, row in loads_df.iterrows():
    # print(_, '->', row)
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0
    if (bus_id in net.bus.name) & (row['phases'] in {'A', 'B', 'C'}):
        if row['phases']=='A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases']=='B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else: # row['phases']=='C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))
        load = pp.create_asymmetric_load(
            net, bus=net.bus.index[net.bus.name==bus_id][0],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
        load_map[load] = row['Yearly']
net

In [ ]:
import pandapower.control as ppc
from pandapower.timeseries.data_sources.frame_data import DFData

ds = DFData(profile_df)

for load_name in net.asymmetric_load.name:
    # get the index of the corresponding load
    idx = net.asymmetric_load[net.asymmetric_load.name == load_name].index[0]

    # find phase from the merged metadata
    phase = merged_load[merged_load["Name_x"] == load_name]["phases"].values[0].lower()

    ppc.ConstControl(
        net, element_index=idx,
        element="asymmetric_load",
        element_name=f"CTRL_{net.asymmetric_load.name.at[idx]}",
        variable=f"p_{phase}_mw",
        profile_name=load_name, data_source=ds
    )
net

In [ ]:
# Create and attach output writer to net48
ow = OutputWriter(net, time_steps=range(profile_df.shape[0]), output_path=output_dir)
# Log desired variables
ow.log_variable('res_trafo_3ph', 'loading_percent', index=net.trafo.index[0])

ow.log_variable('res_bus_3ph', 'vm_a_pu', index=net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_b_pu', index=net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_c_pu', index=net.bus.index)

ow.log_variable('res_line_3ph', 'loading_a_percent', index=net.line.index)
ow.log_variable('res_line_3ph', 'loading_b_percent', index=net.line.index)
ow.log_variable('res_line_3ph', 'loading_c_percent', index=net.line.index)

run_timeseries(net, time_steps=range(profile_df.shape[0]), run=pp.runpp_3ph, run_control=True, continue_on_divergence=True)

In [ ]:
ds

In [ ]:
vm_a_pu = pd.read_pickle(os.path.join(output_dir, 'res_bus_3ph', 'vm_a_pu.p'))
vm_b_pu = pd.read_pickle(os.path.join(output_dir, 'res_bus_3ph', 'vm_b_pu.p'))
vm_c_pu = pd.read_pickle(os.path.join(output_dir, 'res_bus_3ph', 'vm_c_pu.p'))

trafo_loading = pd.read_pickle(os.path.join(output_dir, 'res_trafo_3ph', 'loading_percent.p'))

vm_a_pu.shape, vm_b_pu.shape, vm_c_pu.shape, trafo_loading.shape

In [ ]:
vm_a_pu.to_csv(os.path.join(output_dir, 'vm_a_pu.csv'))
vm_b_pu.to_csv(os.path.join(output_dir, 'vm_b_pu.csv'))
vm_c_pu.to_csv(os.path.join(output_dir, 'vm_c_pu.csv'))
net.res_bus_3ph.to_csv(os.path.join(output_dir, 'net.res_bus_3ph.csv'))
trafo_loading.to_csv(os.path.join(output_dir, 'trafo_loading_percent.csv'))

vm_a_pu.head(20)

In [ ]:
trafo_loading.describe()